**[Intermediate Machine Learning Home Page](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


By encoding **categorical variables**, you'll obtain your best results thus far!

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In this exercise, you will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('train.csv', index_col='Id') 
X_test = pd.read_csv('test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()]
cols_with_missing1 = [col for col in X_test.columns if X_test[col].isnull().any()]
set_a=set(cols_with_missing)
set_b=set(cols_with_missing1)
overall_missing=list(set_a.union(set_b))
X.drop(overall_missing, axis=1, inplace=True)
#X.drop(co_missing1, axis=1, inpalce=True)
X_test.drop(overall_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [59]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 619 to 685
Data columns (total 45 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSSubClass     1168 non-null   int64 
 1   LotArea        1168 non-null   int64 
 2   Street         1168 non-null   object
 3   LotShape       1168 non-null   object
 4   LandContour    1168 non-null   object
 5   LotConfig      1168 non-null   object
 6   LandSlope      1168 non-null   object
 7   Neighborhood   1168 non-null   object
 8   Condition1     1168 non-null   object
 9   Condition2     1168 non-null   object
 10  BldgType       1168 non-null   object
 11  HouseStyle     1168 non-null   object
 12  OverallQual    1168 non-null   int64 
 13  OverallCond    1168 non-null   int64 
 14  YearBuilt      1168 non-null   int64 
 15  YearRemodAdd   1168 non-null   int64 
 16  RoofStyle      1168 non-null   object
 17  RoofMatl       1168 non-null   object
 18  ExterQual      1168 non-nul

Use the next code cell to print the first five rows of the data.

In [60]:
X_train.head()

MSSubClass  LotArea Street LotShape LandContour LotConfig LandSlope  \
Id                                                                         
619          20    11694   Pave      Reg         Lvl    Inside       Gtl   
871          20     6600   Pave      Reg         Lvl    Inside       Gtl   
93           30    13360   Pave      IR1         HLS    Inside       Gtl   
818          20    13265   Pave      IR1         Lvl   CulDSac       Gtl   
303          20    13704   Pave      IR1         Lvl    Corner       Gtl   

    Neighborhood Condition1 Condition2  ... WoodDeckSF OpenPorchSF  \
Id                                      ...                          
619      NridgHt       Norm       Norm  ...          0         108   
871        NAmes       PosN       Norm  ...          0           0   
93       Crawfor       Norm       Norm  ...          0           0   
818      Mitchel       Norm       Norm  ...        150          59   
303      CollgCr       Norm       Norm  ...        468          81   

     EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea MiscVal MoSold YrSold  \
Id                                                                           
619              0          0          260         0       0      7   2007   
871              0          0            0         0       0      8   2009   
93              44          0            0         0       0      8   2009   
818              0          0            0         0       0      7   2008   
303              0          0            0         0       0      1   2006   

    SaleCondition  
Id                 
619       Partial  
871        Normal  
93         Normal  
818        Normal  
303        Normal  

[5 rows x 45 columns]

Notice that the dataset contains both numerical and categorical variables.  You'll need to encode the categorical data before training a model.

To compare different models, you'll use the same `score_dataset()` function from the tutorial.  This function reports the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) from a random forest model.

In [61]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Step 1: Drop columns with categorical data

You'll get started with the most straightforward approach.  Use the code cell below to preprocess the data in `X_train` and `X_valid` to remove columns with categorical data.  Set the preprocessed DataFrames to `drop_X_train` and `drop_X_valid`, respectively.  

In [62]:
# Fill in the lines below: drop columns in training and validation data
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

# Check your answers
#step_1.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_1.hint()
#step_1.solution()

Run the next code cell to get the MAE for this approach.

In [63]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
18934.51899543379


# Step 2: Label encoding

Before jumping into label encoding, we'll investigate the dataset.  Specifically, we'll look at the `'Condition2'` column.  The code cell below prints the unique entries in both the training and validation sets.

In [64]:
X_train.Condition2.unique()

array(['Norm', 'PosA', 'Feedr', 'PosN', 'Artery', 'RRAe'], dtype=object)

In [65]:
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


If you now write code to: 
- fit a label encoder to the training data, and then 
- use it to transform both the training and validation data, 

you'll get an error.  Can you see why this is the case?  (_You'll need  to use the above output to answer this question._)

This is a common problem that you'll encounter with real-world data, and there are many approaches to fixing this issue.  For instance, you can write a custom label encoder to deal with new categories.  The simplest approach, however, is to drop the problematic categorical columns.  

Run the code cell below to save the problematic columns to a Python list `bad_label_cols`.  Likewise, columns that can be safely label encoded are stored in `good_label_cols`.

In [66]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

In [67]:
len(object_cols)

20

In [68]:
X_valid[object_cols].head()

Street LotShape LandContour LotConfig LandSlope Neighborhood Condition1  \
Id                                                                            
530   Pave      IR1         Lvl   CulDSac       Gtl      Crawfor       Norm   
492   Pave      Reg         Lvl    Inside       Gtl        NAmes     Artery   
460   Pave      IR1         Bnk    Corner       Gtl      BrkSide       Norm   
280   Pave      Reg         Lvl    Inside       Gtl      ClearCr       Norm   
656   Pave      Reg         Lvl    Inside       Gtl       BrDale       Norm   

    Condition2 BldgType HouseStyle RoofStyle RoofMatl ExterQual ExterCond  \
Id                                                                          
530       Norm     1Fam     1Story       Hip  CompShg        Gd        TA   
492       Norm     1Fam     1.5Fin     Gable  CompShg        TA        TA   
460       Norm     1Fam     1.5Fin     Gable  CompShg        TA        TA   
280       Norm     1Fam     2Story       Hip  CompShg        TA        TA   
656       Norm    Twnhs     2Story     Gable  CompShg        TA        TA   

    Foundation Heating HeatingQC CentralAir PavedDrive SaleCondition  
Id                                                                    
530      PConc    GasA        TA          Y          Y        Alloca  
492     CBlock    GasA        TA          Y          Y        Normal  
460     CBlock    GasA        TA          Y          Y        Normal  
280     CBlock    GasA        Ex          Y          Y        Normal  
656     CBlock    GasA        TA          Y          Y        Family

In [69]:
X_train[object_cols].head()

Street LotShape LandContour LotConfig LandSlope Neighborhood Condition1  \
Id                                                                            
619   Pave      Reg         Lvl    Inside       Gtl      NridgHt       Norm   
871   Pave      Reg         Lvl    Inside       Gtl        NAmes       PosN   
93    Pave      IR1         HLS    Inside       Gtl      Crawfor       Norm   
818   Pave      IR1         Lvl   CulDSac       Gtl      Mitchel       Norm   
303   Pave      IR1         Lvl    Corner       Gtl      CollgCr       Norm   

    Condition2 BldgType HouseStyle RoofStyle RoofMatl ExterQual ExterCond  \
Id                                                                          
619       Norm     1Fam     1Story       Hip  CompShg        Ex        TA   
871       Norm     1Fam     1Story       Hip  CompShg        TA        TA   
93        Norm     1Fam     1Story     Gable  CompShg        TA        Gd   
818       Norm     1Fam     1Story       Hip  CompShg        Gd        TA   
303       Norm     1Fam     1Story     Gable  CompShg        Gd        TA   

    Foundation Heating HeatingQC CentralAir PavedDrive SaleCondition  
Id                                                                    
619      PConc    GasA        Ex          Y          Y       Partial  
871     CBlock    GasA        Gd          N          Y        Normal  
93      BrkTil    GasA        Ex          Y          Y        Normal  
818      PConc    GasA        Ex          Y          Y        Normal  
303      PConc    GasA        Ex          Y          Y        Normal

In [70]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['LandSlope', 'RoofMatl', 'ExterCond', 'Heating', 'Neighborhood', 'RoofStyle', 'Condition1', 'Foundation', 'HeatingQC', 'Condition2']


Use the next code cell to label encode the data in `X_train` and `X_valid`.  Set the preprocessed DataFrames to `label_X_train` and `label_X_valid`, respectively.  
- We have provided code below to drop the categorical columns in `bad_label_cols` from the dataset. 
- You should label encode the categorical columns in `good_label_cols`.  

In [71]:
from sklearn.preprocessing import LabelEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
label_encoder = LabelEncoder()
for col in set(good_label_cols):
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])
# Your code here
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))    
# Check your answer
#step_2.b.check()

MAE from Approach 2 (Label Encoding):
18232.20285958904


In [72]:
label_X_train.head()

MSSubClass  LotArea  Street  LotShape  LandContour  LotConfig  BldgType  \
Id                                                                             
619          20    11694       1         3            3          4         0   
871          20     6600       1         3            3          4         0   
93           30    13360       1         0            1          4         0   
818          20    13265       1         0            3          1         0   
303          20    13704       1         0            3          0         0   

     HouseStyle  OverallQual  OverallCond  ...  WoodDeckSF  OpenPorchSF  \
Id                                         ...                            
619           2            9            5  ...           0          108   
871           2            5            5  ...           0            0   
93            2            5            7  ...           0            0   
818           2            8            5  ...         150           59   
303           2            7            5  ...         468           81   

     EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  \
Id                                                                              
619              0          0          260         0        0       7    2007   
871              0          0            0         0        0       8    2009   
93              44          0            0         0        0       8    2009   
818              0          0            0         0        0       7    2008   
303              0          0            0         0        0       1    2006   

     SaleCondition  
Id                  
619              5  
871              4  
93               4  
818              4  
303              4  

[5 rows x 35 columns]

In [73]:
X_train.head()

MSSubClass  LotArea Street LotShape LandContour LotConfig LandSlope  \
Id                                                                         
619          20    11694   Pave      Reg         Lvl    Inside       Gtl   
871          20     6600   Pave      Reg         Lvl    Inside       Gtl   
93           30    13360   Pave      IR1         HLS    Inside       Gtl   
818          20    13265   Pave      IR1         Lvl   CulDSac       Gtl   
303          20    13704   Pave      IR1         Lvl    Corner       Gtl   

    Neighborhood Condition1 Condition2  ... WoodDeckSF OpenPorchSF  \
Id                                      ...                          
619      NridgHt       Norm       Norm  ...          0         108   
871        NAmes       PosN       Norm  ...          0           0   
93       Crawfor       Norm       Norm  ...          0           0   
818      Mitchel       Norm       Norm  ...        150          59   
303      CollgCr       Norm       Norm  ...        468          81   

     EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea MiscVal MoSold YrSold  \
Id                                                                           
619              0          0          260         0       0      7   2007   
871              0          0            0         0       0      8   2009   
93              44          0            0         0       0      8   2009   
818              0          0            0         0       0      7   2008   
303              0          0            0         0       0      1   2006   

    SaleCondition  
Id                 
619       Partial  
871        Normal  
93         Normal  
818        Normal  
303        Normal  

[5 rows x 45 columns]

In [ ]:
# Lines below will give you a hint or solution code
step_2.b.hint()
step_2.b.solution()

Run the next code cell to get the MAE for this approach.

In [74]:
print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Label Encoding):
18232.20285958904


# Step 3: Investigating cardinality

So far, you've tried two different approaches to dealing with categorical variables.  And, you've seen that encoding categorical data yields better results than removing columns from the dataset.

Soon, you'll try one-hot encoding.  Before then, there's one additional topic we need to cover.  Begin by running the next code cell without changes.  

In [75]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('Neighborhood', 25)]

The output above shows, for each column with categorical data, the number of unique values in the column.  For instance, the `'Street'` column in the training data has two unique values: `'Grvl'` and `'Pave'`, corresponding to a gravel road and a paved road, respectively.

We refer to the number of unique entries of a categorical variable as the **cardinality** of that categorical variable.  For instance, the `'Street'` variable has cardinality 2.

Use the output above to answer the questions below.

In [ ]:
# Fill in the line below: How many categorical variables in the training data
# have cardinality greater than 10?
high_cardinality_numcols = 3

# Fill in the line below: How many columns are needed to one-hot encode the 
# 'Neighborhood' variable in the training data?
num_cols_neighborhood = 25

# Check your answers
step_3.a.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_3.a.hint()
step_3.a.solution()

For large datasets with many rows, one-hot encoding can greatly expand the size of the dataset.  For this reason, we typically will only one-hot encode columns with relatively low cardinality.  Then, high cardinality columns can either be dropped from the dataset, or we can use label encoding.

As an example, consider a dataset with 10,000 rows, and containing one categorical column with 100 unique entries.  
- If this column is replaced with the corresponding one-hot encoding, how many entries are added to the dataset?  
- If we instead replace the column with the label encoding, how many entries are added?  

Use your answers to fill in the lines below.

In [ ]:
# Fill in the line below: How many entries are added to the dataset by 
# replacing the column with a one-hot encoding?
OH_entries_added = 1e4*100-1e4

# Fill in the line below: How many entries are added to the dataset by
# replacing the column with a label encoding?
label_entries_added = 0

# Check your answers
step_3.b.check()

In [ ]:
# Lines below will give you a hint or solution code
step_3.b.hint()
step_3.b.solution()

# Step 4: One-hot encoding

In this step, you'll experiment with one-hot encoding.  But, instead of encoding all of the categorical variables in the dataset, you'll only create a one-hot encoding for columns with cardinality less than 10.

Run the code cell below without changes to set `low_cardinality_cols` to a Python list containing the columns that will be one-hot encoded.  Likewise, `high_cardinality_cols` contains a list of categorical columns that will be dropped from the dataset.

In [76]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Neighborhood']


Use the next code cell to one-hot encode the data in `X_train` and `X_valid`.  Set the preprocessed DataFrames to `OH_X_train` and `OH_X_valid`, respectively.  
- The full list of categorical columns in the dataset can be found in the Python list `object_cols`.
- You should only one-hot encode the categorical columns in `low_cardinality_cols`.  All other categorical columns should be dropped from the dataset. 

In [77]:
from sklearn.preprocessing import OneHotEncoder
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Check your answer
#step_4.check()

In [ ]:
# Lines below will give you a hint or solution code
step_4.hint()
step_4.solution()

Run the next code cell to get the MAE for this approach.

In [78]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
18550.40561643836


# Step 5: Generate test predictions and submit your results

After you complete Step 4, if you'd like to use what you've learned to submit your results to the leaderboard, you'll need to preprocess the test data before generating predictions.

**This step is completely optional, and you do not need to submit results to the leaderboard to successfully complete the exercise.**

Check out the previous exercise if you need help with remembering how to [join the competition](https://www.kaggle.com/c/home-data-for-ml-course) or save your results to CSV.  Once you have generated a file with your results, follow the instructions below:
- Begin by clicking on the blue **COMMIT** button in the top right corner.  This will generate a pop-up window.  
- After your code has finished running, click on the blue **Open Version** button in the top right of the pop-up window.  This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
- Click on the **Output** tab on the left of the screen.  Then, click on the **Submit to Competition** button to submit your results to the leaderboard.
- If you want to keep working to improve your performance, select the blue **Edit** button in the top right of the screen. Then you can change your model and repeat the process.

In [38]:
# function for predicting X_test
def predict_dataset(X_train, X_test, y_train):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    df=pd.DataFrame(data=preds,index=X_test.index,columns=['SalePrice'])
    df.to_csv('RFR_results.csv',index=True)
    

In [79]:
print(X_test.shape)
print(X_train.shape)

(1459, 45)
(1168, 45)


In [80]:
drop_X_test = X_test.select_dtypes(exclude=['object'])

In [81]:
# All categorical columns
object_cols = [col for col in X_test.columns if X_test[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_test[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['RoofMatl', 'Heating', 'HouseStyle', 'Condition2']


In [98]:
# Drop categorical columns that will not be encoded
label_X_test = X_test.drop(bad_label_cols, axis=1)
label_X_train=X_train.drop(bad_label_cols, axis=1)

In [99]:
# Apply label encoder 
label_encoder = LabelEncoder()
for col in set(good_label_cols):
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_test[col] = label_encoder.transform(X_test[col])

In [100]:
print(label_X_train.shape)
print(label_X_test.shape)

(1168, 41)
(1459, 41)


In [101]:
predict_dataset(label_X_train,label_X_test,y_train)

# Keep going

With missing value handling and categorical encoding, your modeling process is getting complex. This complexity gets worse when you want to save your model to use in the future. The key to managing this complexity is something called **pipelines**. 

**[Learn to use pipelines](https://www.kaggle.com/alexisbcook/pipelines)** to preprocess datasets with categorical variables, missing values and any other messiness your data throws at you.

---
**[Intermediate Machine Learning Home Page](https://www.kaggle.com/learn/intermediate-machine-learning)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*